In [1]:
from icap.database.icapdatabase import ICapDatabase
from icap.results.results import Results
from icap.peco.peco import PECORecipe, PECO, PECOInterval, PECODemand

import pandas as pd
import numpy as np
import datetime

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

In [3]:
p = PECO(conn)
pint = PECOInterval(conn)
#pint = PECODemand(conn)
pint.compute_icap().head()

,RunDate,ISO,Utility,PremiseId,Year,RateClass,Strata,MeterType,ICap
0,2018-08-25 09:38:56.443260,PJM,PECO,0029022099,2018,GS,101,INT,252.180303
1,2018-08-25 09:38:56.443260,PJM,PECO,0029022099,2018,GS,101,INT,229.172994
2,2018-08-25 09:38:56.443260,PJM,PECO,0029022099,2018,GS,101,INT,247.628833
3,2018-08-25 09:38:56.443260,PJM,PECO,0029022099,2018,GS,101,INT,261.181601
4,2018-08-25 09:38:56.443260,PJM,PECO,0029022099,2018,GS,101,INT,228.450782


In [4]:
pint.write_nits()

In [4]:
pint.icap_df_.head()

,PremiseId,UsageDate,HourEnding,Usage,Year,RateClass,Strata,CPDate,ParameterId,NCRatio,WCF,WCFMean,Count,RCLF,NSPLScale,ICap,NITS
0,0029022099,2017-06-13,17.0,221.952,2017,GS,101,2017-06-13,NCRatio,1.03,228.610554,220.943615,5.0,1.1031,1.030154,243.722903,251.07215
1,0029022099,2017-07-21,17.0,218.688,2017,GS,101,2017-07-21,NCRatio,0.95,207.753597,220.943615,5.0,1.1031,1.030154,243.722903,251.07215
2,0029022099,2017-07-20,17.0,226.752,2017,GS,101,2017-07-20,NCRatio,0.99,224.484482,220.943615,5.0,1.1031,1.030154,243.722903,251.07215
3,0029022099,2017-07-19,17.0,232.128,2017,GS,101,2017-07-19,NCRatio,1.02,236.770556,220.943615,5.0,1.1031,1.030154,243.722903,251.07215
4,0029022099,2017-06-12,17.0,213.504,2017,GS,101,2017-06-12,NCRatio,0.97,207.098886,220.943615,5.0,1.1031,1.030154,243.722903,251.07215


In [ ]:
def compute_wcf(group):
    return np.mean(group['WCF'])


wcf = pd.DataFrame(pint.factors_.groupby(['PremiseId', 'Year']).apply(compute_wcf)).reset_index()
wcf = wcf.rename(columns={0:'WCFMean'})
tmp = pd.merge(pint.factors_, wcf,
         left_on=['PremiseId', 'Year'],
         right_on=['PremiseId', 'Year'])

tmp['Year'] = (tmp['Year'].apply(int) + 1).apply(str)
tmp.head()

In [ ]:
pint.icap_df_.columns

In [ ]:
tmp.columns

In [ ]:
keep_cols = ['PremiseId', 'RateClass', 'Strata', 'Year', 'CPDate', 'HourEnding', 'Usage', 'NCRatio', 'WCF_x', 'WCFMean', 'Count', 'ParameterValue', 'ICap', 'RunDate', 'MeterType']
final = pd.merge(tmp, pint.icap_df_,
         left_on=['PremiseId', 'RateClass', 'Strata', 'Year'],
         right_on=['PremiseId', 'RateClass', 'Strata', 'Year'])[keep_cols]

final = final.rename(columns={'WCF_x':'WCF', 'ParameterValue':'RateClassLoss'})

final.columns

In [ ]:
g = pint.records_.groupby('PremiseId')

In [ ]:
final.groupby('PremiseId').apply(len).mean()

In [ ]:
final.columns.shape

In [ ]:
pint.records_['PremiseId'].drop_duplicates().count()

In [ ]:
def compute_wcf(group):
    return np.mean(group['WCF'])


wcf = pd.DataFrame(pint.factors_.groupby(['PremiseId', 'Year']).apply(compute_wcf)).reset_index()
wcf = wcf.rename(columns={0:'WCFMean'})
tmp = pd.merge(pint.factors_, wcf,
         left_on=['PremiseId', 'Year'],
         right_on=['PremiseId', 'Year'])

tmp['Year'] = (tmp['Year'].apply(int) + 1).apply(str)
#tmp.head()


keep_cols = ['PremiseId', 'RateClass', 'Strata', 'Year', 'CPDate', 'HourEnding', 'Usage', 'NCRatio', 'WCF_x', 'WCFMean', 'Count', 'ParameterValue', 'ICap', 'RunDate', 'MeterType']
final = pd.merge(tmp, pint.icap_df_,
         left_on=['PremiseId', 'RateClass', 'Strata', 'Year'],
         right_on=['PremiseId', 'RateClass', 'Strata', 'Year'])[keep_cols]

final = final.rename(columns={'WCF_x':'WCF', 'ParameterValue':'RateClassLoss'})

#final.columns

def write_to_csv(df):
    
    with open('/tmp/peco_interval.csv', 'w') as fout:
        # Header
        fout.write('PremiseId, RateClass, Strata, Year, RunDate,\
        Usage Date 1 (CP 1), Hour Ending 1 (CP 1), CP 1 Usage, NCRatio, WCF,\
        Usage Date 2 (CP 2), Hour Ending 2 (CP 2), CP 2 Usage, NCRatio, WCF,\
        Usage Date 3 (CP 3), Hour Ending 3 (CP 3), CP 3 Usage, NCRatio, WCF,\
        Usage Date 4 (CP 4), Hour Ending 4 (CP 4), CP 4 Usage, NCRatio, WCF,\
        Usage Date 5 (CP 5), Hour Ending 5 (CP 5), CP 5 Usage, NCRatio, WCF,\
        Weather Correction, Rate Class Loss, Meter Type Calculated, Capacity Planning Year, Capacity Tag Value\n')

        # Loop the groups
        for name, group in df.groupby(['PremiseId']):
            
            # If group is missing rows, force empty rows for valid print lengths
            if group.shape[0] != 5:
                num_empties = 5- group.shape[0]
                empty_row = pd.Series([np.nan for col in group.columns], index=group.columns)
                
                for _ in range(num_empties):
                    group = group.append(empty_row, ignore_index=True)

            # Updated group shape (should be 5)        
            last_row = group.shape[0] - 1
            row_number = 0
            current_row = ''
            # Build string output for each group
            for row in group.sort_values(by=['CPDate']).itertuples():
                # Unpack the row
                index, premise_id, rate_class, strata, year, usage_date, hour_ending, usage, nc_ratio, wcf, wcf_mean, rec_count, rclf, icap, run_date, meter_type = row
                
                # Write initial fields
                if row_number == 0:
                    current_row = '{},{},{},{},{},'.format(
                        premise_id, rate_class, strata, year, run_date)
                
                # Build repeated fields
                current_row += '{},{},{},{},{},'.format(usage_date, hour_ending, usage, nc_ratio, wcf)
                
                # Final fields
                if row_number == last_row:
                    current_row += '{},{},{},{},{}\n'.format(wcf_mean, rclf, meter_type, year, icap)        

                row_number += 1
                
            # Write current group
            fout.write(current_row)


In [ ]:
write_to_csv(final)

# NITS

In [5]:
nspl_scale = pint.sys_df_[pint.sys_df_['ParameterId'] == 'TransLoadScaleFactor']
nspl_scale = nspl_scale.rename(columns={'ParameterValue': 'NSPLScale'}).drop(labels=['ParameterId'], axis=1)

In [6]:
# Extract parameter values
nc_ratio = pint.util_df_[pint.util_df_['ParameterId'] == 'NCRatio'].copy()
rclf = pint.util_df_[pint.util_df_['ParameterId'] == 'RateClassLoss']
util_min = pd.merge(nc_ratio, rclf, on=['Year', 'CPDate', 'RateClass', 'Strata'], how='left').fillna(method='ffill')

# Join hourly records and parameter values
_plc = (pd.merge(pint.records_, util_min,
        left_on=['UsageDate', 'RateClass', 'Strata', 'Year'],
        right_on=['CPDate', 'RateClass', 'Strata', 'Year'],
               how='left') 
        .rename(columns={'ParameterValue_x':'NCRatio', 'ParameterValue_y':'RCLF'})
        .drop(labels=['ParameterId_x', 'ParameterId_y'], axis=1))
plc = pd.merge(_plc, nspl_scale, on=['Year'])

# Compute
plc['PLC_factor'] = plc.Usage * plc.NCRatio * plc.RCLF
plc['PLC'] = plc.groupby(['PremiseId', 'Year'])['PLC_factor'].transform(np.mean)
plc['NITS'] = plc.PLC * plc.NSPLScale

In [12]:
plc[plc['PremiseId'] == '0029022099']

,PremiseId,UsageDate,HourEnding,Usage,Year,RateClass,Strata,CPDate,NCRatio,RCLF,NSPLScale,PLC_factor,PLC,NITS
0,0029022099,2017-06-13,17.0,221.952,2017,GS,101,2017-06-13,1.03,1.1031,1.030154,252.180303,243.722903,251.07215
1,0029022099,2017-07-21,17.0,218.688,2017,GS,101,2017-07-21,0.95,1.1031,1.030154,229.172994,243.722903,251.07215
2,0029022099,2017-07-20,17.0,226.752,2017,GS,101,2017-07-20,0.99,1.1031,1.030154,247.628833,243.722903,251.07215
3,0029022099,2017-07-19,17.0,232.128,2017,GS,101,2017-07-19,1.02,1.1031,1.030154,261.181601,243.722903,251.07215
4,0029022099,2017-06-12,17.0,213.504,2017,GS,101,2017-06-12,0.97,1.1031,1.030154,228.450782,243.722903,251.07215


In [9]:
pint_icap = pint.compute_icap()

In [ ]:
_tmp.head()

In [ ]:
plc.head()

In [14]:
def write_to_csv(df, meter_type):
    
    -


In [15]:
write_to_csv(plc, 'INT')